In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
batch_size=200
learning_rate=0.01
epochs=10

# 加载数据
train_db=datasets.MNIST('./data/mnist_data',train=True,download=True,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ]))
# train_loader=torch.utils.data.DataLoader(train_db,batch_size=batch_size,shuffle=True)

test_db=datasets.MNIST('./data/mnist_data',train=False,
                  transform=transforms.Compose([
                      transforms.ToTensor(),
                      transforms.Normalize((0.1307,),(0.3081,))
                  ]))
test_loader=torch.utils.data.DataLoader(test_db,batch_size=batch_size,shuffle=True)
print('train:',len(train_db),'test:',len(test_db))

train: 60000 test: 10000


In [3]:
train_db,val_db=torch.utils.data.random_split(train_db,[50000,10000])
print('db1:',len(train_db),'db2:',len(val_db))

train_loader=torch.utils.data.DataLoader(train_db,batch_size=batch_size,shuffle=True)
val_loader=torch.utils.data.DataLoader(val_db,batch_size=batch_size,shuffle=True)

db1: 50000 db2: 10000


In [5]:
class MLP(nn.Module):
    
    def __init__(self):
        super(MLP,self).__init__()
        
        self.model=nn.Sequential(
            nn.Linear(784,200),
            nn.ReLU(inplace=True),
            nn.Linear(200,200),
            nn.ReLU(inplace=True),
            nn.Linear(200,10),
            nn.ReLU(inplace=True),
        )
        
    def forward(self,x):
        x=self.model(x)
        return x

In [6]:
device=torch.device('cuda:0')
net=MLP().to(device)
optimizer=optim.SGD(net.parameters(),lr=learning_rate)
criteon=nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):
    
    for batch_idx,(data,target) in enumerate(train_loader):
        data=data.view(-1,28*28)
        data,target=data.to(device),target.to(device)
        
        logits=net(data)
        loss=criteon(logits,target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 ==0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            
    val_loss=0 
    correct=0 
    for data,target in val_loader:
        data=data.view(-1,28*28)
        data,target=data.to(device),target.to(device)
        
        logits=net(data)
        val_loss+=criteon(logits,target).item()
        
        pred=logits.argmax(dim=1)
        correct+=pred.eq(target.data).sum()
        
    val_loss /= len(val_loader.dataset)
    print('Val set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.296748
Train Epoch: 0 [20000/50000 (40%)]	Loss: 1.949544
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.147922

Val set: Average loss: 0.0045, Accuracy: 8050/10000 (80%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 0.854956
Train Epoch: 1 [20000/50000 (40%)]	Loss: 0.673033
Train Epoch: 1 [40000/50000 (80%)]	Loss: 0.542316

Val set: Average loss: 0.0024, Accuracy: 8673/10000 (87%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.473701
Train Epoch: 2 [20000/50000 (40%)]	Loss: 0.369894
Train Epoch: 2 [40000/50000 (80%)]	Loss: 0.349003

Val set: Average loss: 0.0020, Accuracy: 8866/10000 (89%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.381842
Train Epoch: 3 [20000/50000 (40%)]	Loss: 0.418252
Train Epoch: 3 [40000/50000 (80%)]	Loss: 0.379919

Val set: Average loss: 0.0018, Accuracy: 8965/10000 (90%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.320078
Train Epoch: 4 [20000/50000 (40%)]	Loss: 0.246844
Train Epoch: 4 [40000/50000 (80%)]	Loss: 0.350632

Val set: Average loss: 0.0017, Acc

In [7]:
test_loss = 0
correct = 0
for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.cuda()
    logits = net(data)
    test_loss += criteon(logits, target).item()

    pred = logits.data.max(1)[1]
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0012, Accuracy: 9320/10000 (93%)

